In [15]:
%load_ext memory_profiler
%load_ext line_profiler
import pyxacc as xacc
xacc.Initialize(['--compiler','quil'])
import random
import numpy as np

def random_circuit(n_qbits, ninsts, nearestNeighbor = True):
    quilStr = '__qpu__ f(AcceleratorBuffer b) {\n'
    for i in range(ninsts):
        gatetype = random.randint(1,2)
        instr = None
        if gatetype==1:
            gatenames = ['H','X','Y','Z','RX','RY','RZ']
            iqbit = random.randint(0,n_qbits-1)

            gate = random.choice(gatenames)
            if 'R' in gate:
                angle = str(np.random.uniform(0,np.pi));
                instr = gate + '('+angle+') %d'% iqbit
            else:
                instr = gate + ' %d'% iqbit
        if gatetype==2:
            gatenames = ['CNOT']
            iqbits = random.sample(range(n_qbits),2)
            plusoneminusone = 1 if random.randint(1,2) == 1 else -1
            if plusoneminusone == 1 and iqbits[0]+1 == n_qbits:
                plusoneminusone = -1
            if plusoneminusone == -1 and iqbits[0] == 0:
                plusoneminusone = 1
            instr = random.choice(gatenames)+' %d %d'% (iqbits[0],iqbits[0]+plusoneminusone if nearestNeighbor else iqbits[1] )
        quilStr += instr+'\n'
    quilStr += '}'
    return quilStr


The memory_profiler extension is already loaded. To reload it, use:
  %reload_ext memory_profiler
The line_profiler extension is already loaded. To reload it, use:
  %reload_ext line_profiler


In [20]:
qpu = xacc.getAccelerator('tnqvm')
xacc.setOption('tnqvm-verbose','')
for nq in list(range(5,105, 5)):
    circ = random_circuit(nq,200)
    ncnots = circ.count('CNOT')
    f = xacc.compileKernel(qpu, circ)
    buf = qpu.createBuffer('q',nq)
    print('NQubits = ', nq, ', NCNOTS = ', ncnots, ', ', end='')
    %memit qpu.execute(buf, f)


NQubits =  5 , NCNOTS =  87 , peak memory: 129.36 MiB, increment: 0.00 MiB
NQubits =  10 , NCNOTS =  102 , peak memory: 129.36 MiB, increment: 0.00 MiB
NQubits =  15 , NCNOTS =  87 , peak memory: 129.36 MiB, increment: 0.00 MiB
NQubits =  20 , NCNOTS =  102 , peak memory: 129.36 MiB, increment: 0.00 MiB
NQubits =  25 , NCNOTS =  106 , peak memory: 129.36 MiB, increment: 0.00 MiB
NQubits =  30 , NCNOTS =  99 , peak memory: 129.36 MiB, increment: 0.00 MiB
NQubits =  35 , NCNOTS =  101 , peak memory: 129.36 MiB, increment: 0.00 MiB
NQubits =  40 , NCNOTS =  101 , peak memory: 129.36 MiB, increment: 0.00 MiB
NQubits =  45 , NCNOTS =  97 , peak memory: 129.36 MiB, increment: 0.00 MiB
NQubits =  50 , NCNOTS =  97 , peak memory: 129.36 MiB, increment: 0.00 MiB
NQubits =  55 , NCNOTS =  93 , peak memory: 129.36 MiB, increment: 0.00 MiB
NQubits =  60 , NCNOTS =  104 , peak memory: 129.36 MiB, increment: 0.00 MiB
NQubits =  65 , NCNOTS =  99 , peak memory: 129.36 MiB, increment: 0.00 MiB
NQubits

In [18]:

for nq in list(range(5,30, 5)):
    circ = random_circuit(nq,100, False)
    ncnots = circ.count('CNOT')
    f = xacc.compileKernel(qpu, circ)
    buf = qpu.createBuffer('q',nq)
    print('NQubits = ', nq, ', NCNOTS = ', ncnots, ', ', end='')
    %memit qpu.execute(buf, f)


NQubits =  5 , NCNOTS =  41 , peak memory: 128.72 MiB, increment: 0.00 MiB
NQubits =  10 , NCNOTS =  48 , peak memory: 128.72 MiB, increment: 0.00 MiB
NQubits =  15 , NCNOTS =  55 , peak memory: 128.72 MiB, increment: 0.00 MiB
NQubits =  20 , NCNOTS =  54 , peak memory: 137.60 MiB, increment: 8.88 MiB
NQubits =  25 , NCNOTS =  63 , peak memory: 129.36 MiB, increment: 0.00 MiB
